# Inference

Because this competition has a hidden test set - we need to set up inference note-book in order to submit our models. 

# Prerequisite 

The additional data have to be added via kaggle's dataset 

- Petfinder data (test data is unavailable) 
- attrdictw wheels (../input/attrdictw/attrdict-2.0.1-py2.py3-none-any.whl) 
- pretrained-model's pth weights (because internet is not available) (../input/resnet34/resnet34-43635321.pth) 
- timmaster (for timm ) 

# Load libraries 

No need for pip because we installed them via dataset ^

In [1]:
! pip install ../input/attrdictw/attrdict-2.0.1-py2.py3-none-any.whl

Processing /kaggle/input/attrdictw/attrdict-2.0.1-py2.py3-none-any.whl


In [2]:
import sys
sys.path.append("../input/timmmaster/")

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
import os
import tqdm

import seaborn as sns
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision.utils import make_grid
from attrdict import AttrDict
import torch
import yaml
from sklearn.model_selection import StratifiedKFold
import copy
import pickle
# from tqdm import tqdm_notebook

# additional lightning 

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule


# pytorch
import torch
from torch import nn
import torch.nn.functional as F
from timm import create_model

In [4]:
class pawnetDataset(torch.utils.data.Dataset):
    """
    Dataset
    Based on template https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
    """
    def __init__(self,annotation_df, img_dir,transform=None,target_transform=None,test=False,custom_len=None):
        self.annotation_df = annotation_df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.test=test # if dataset contains labels
        self.custom_len=custom_len # if we want to define our own epoch
        
        
    def __len__(self):
        """Define 1 epoch"""
        if self.custom_len is None:
            return len(self.annotation_df)
        else:
            return self.custom_len
    
    def __getitem__(self,idx):
        """called batch num of times"""
        img_path = os.path.join(self.img_dir, self.annotation_df.iloc[idx, 0]) # ID is column index 0
        image = read_image(img_path+".jpg")
        if self.test:
            label = 0
        else:
            label = self.annotation_df.iloc[idx, 13] # Pawpularity is column index 13
            
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

class pawNetBasic(pl.LightningModule):
    """
    First cut basic pawNet model
    we will improve on this - this serves as skeleton code
    for other models
    
    timm contains collection of several pretrained models
    
    This is a lightning variant *
    
    
    lightning model requires the following methods:
    1. forward 
    2. training_step (logic inside the iteration loop) , validation_step, test_step (not stable on tpu)
    3. training_epoch_end, validation_epoch_end
    4. configure_optimizers 
    
    other configurable list here https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html
    
    """
    
    def __init__(self,criterion, dropout=0.2):
        super().__init__()
        self.dropout = 0.2
        self._criterion = criterion
        self.train_loss = []
        self.train_rmse = []
        self.valid_rmse = []
        
        # initialize layers
        # https://fastai.github.io/timmdocs/tutorial_feature_extractor
        # remove FCL by setting num_classes=0
        # removed prelogit as well as head (final linear layer)
        self.pretrained = create_model(
#             base_config_manager.load_config().model.pretrained, 
            # todo: add to config 
            # "vit_large_patch32_224_in21k", # out of memory in kaggle kernel
            #"vit_large_patch16_224_in21k", # out of memory
            #"vit_base_patch16_224_in21k", # some error after 2 hours?
            "vit_tiny_patch16_224",
            pretrained=True, 
            num_classes=0, 
            in_chans=3
        )
#         self.global_avg_pooling = torch.nn.AdaptiveAvgPool2d(1) # timm pretrain comes with pooling
        self.linear_1 = torch.nn.Linear(in_features=192,out_features=1000)
        self.prelu = torch.nn.PReLU()
        self.linear_2 = torch.nn.Linear(in_features=1000,out_features=1)
        
    def forward(self,x):
        out = self.pretrained(x)
#         out = out.view(out.size(0), -1) # reshape for linear. timm already returns with CHW flatten
        out = torch.nn.Dropout(self.dropout)(out)
        out = self.linear_1(out)
        out = self.prelu(out)
        out = self.linear_2(out)
        
        
        
        return out
    
    
    def training_step(self, batch, batch_idx):
        """
        logic instead batch loop
        """
        loss, pred, labels = self.__share_step(batch, 'train')
        
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        """
        logic instead batch loop for validation
        """
        
        loss, pred, labels = self.__share_step(batch, 'val')
        return {'loss': loss, 'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, labels = batch
        labels = labels.float() / 100.0
        
        logits = self.forward(images).squeeze(1)
        loss = self._criterion(logits, labels)
        
        # return logloss for training mode, scaled for others
        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        return loss, pred, labels
        
    def training_epoch_end(self, outputs):
        """
        called every end of epoch, contains logic
        at end of epoch
        """
        self.__share_epoch_end(outputs, mode = 'train')

    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, mode = 'val')    
        
    def __share_epoch_end(self, outputs, mode):
        """
        output is a list of output defined in
        `training_step` as well as `validation_step`.
        Need to iterate through each iteration's output.
        the output was a dict
        """
        preds = []
        labels = []
        losses = []
        for out in outputs:
            pred, label, loss = out['pred'], out['labels'], out["loss"]
            preds.append(pred)
            labels.append(label)
            losses.append(loss.view(-1,1))
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        losses = torch.cat(losses)
        if mode == "train":
            loglogss_metrics = losses.mean() # average logloss across iterations
            self.log(f'{mode}_logloss', loglogss_metrics, prog_bar=True)
        else:
            print(f"{mode}: skip logging for logloss")
            
        # RMSE
        metrics = torch.sqrt(((labels - preds) ** 2).mean())
        # https://pytorch-lightning.readthedocs.io/en/stable/extensions/logging.html
        # automatic accumulation at end of epoch for training, true always for test,validation loops
        self.log(f'{mode}_RMSE_loss', metrics, prog_bar=True)
        
        
    def configure_optimizers(self):
        """
        https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.core.lightning.html#pytorch_lightning.core.lightning.LightningModule.configure_optimizers
        
        Any of these 6 options.

        Single optimizer.

        List or Tuple of optimizers.

        Two lists - The first list has multiple optimizers, and the second has multiple LR schedulers (or multiple lr_scheduler_config).

        Dictionary, with an "optimizer" key, and (optionally) a "lr_scheduler" key whose value is a single LR scheduler or lr_scheduler_config.

        Tuple of dictionaries as described above, with an optional "frequency" key.

        None - Fit will run without any optimizer.
        """
        #opt = torch.optim.Adam(self.parameters())
        # TODO: add learning rate to config
        opt = torch.optim.AdamW(self.parameters(),lr=1e-5)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt,T_0=20,eta_min=1e-4)
  
        return [opt]



def inference_test(model,valid_loader,criterion,device= "cpu"):
    """
    performs inference for submission. Note that because
    this is test, there is no actual labels
    """
    model.eval()
    y_valid = []
    y_pred_valid = []
    for i, (x,y) in enumerate(valid_loader):
        with torch.no_grad():
            pred = model(x.to(device))
            pred = torch.sigmoid(pred) * 100.
            y_pred_valid.append(pred.squeeze().detach().cpu())
            y_valid.append(y.detach().cpu())
    # convert from list to tensor
    y_valid = torch.cat(y_valid,0)
    y_pred_valid = torch.cat(y_pred_valid,0)
    if criterion is None:
        valid_loss = None
    else:
        
        valid_loss = criterion(y_pred_valid,y_valid).item()
    
    return valid_loss,y_pred_valid

In [5]:
"""
To add to utility.py
"""

# def seed_everything(seed=1234):
#     """
#     Utility function to seed everything
#     source: https://www.kaggle.com/bminixhofer/deterministic-neural-networks-using-pytorch
#     """
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True

    

def read_yaml(filename):
    """
    Read yaml configuation and returns the dict

    Parameters
    ----------
    filename: string
        Path including yaml file name
    """

    with open(filename) as f:
        config = yaml.safe_load(f)

    return config


    
# configs

# config is different in kaggle


class BaseConfigLoader:
    
    def __init__(self,config_path):
        self.config = read_yaml(config_path)
            
    def load_config(self):
        return AttrDict(self.config)

# Load test data 

In [6]:
# this is specific to kaggle
# if running in GCS, replace with our GCP bucket 
# get cache location of the dataset 
# GCS_DS_PATH = KaggleDatasets().get_gcs_path()
base_config_manager = BaseConfigLoader("../input/config/config.yaml")
file_path = base_config_manager.load_config().filepath.kaggle #"/kaggle/input/petfinder-pawpularity-score/"
test_df = pd.read_csv(os.path.join(file_path,"test.csv"))

In [7]:
test_transformation = T.Compose([
                T.Resize([224,224]),# imgnet needs at least 224
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ), # imgnet requirements 
                ]
            )


test_data = pawnetDataset(annotation_df=test_df,img_dir =os.path.join(file_path,"test"), transform = test_transformation,test=True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=64,shuffle=False,num_workers=2)

# Load weights and inference

https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html

In [8]:
# create model and load checkpoint 

# load config
# this object manages all the configurations

# base_config_manager = BaseConfigLoader("../input/config/config.yaml")
criterion = torch.nn.BCEWithLogitsLoss()

In [9]:
# load and weights - will fail hre because no internet
try:
    model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"../input/vit-small/pawnet_lightning_vit_tiny_patch16_224/default/version_0/checkpoints/best_loss.ckpt",criterion=criterion)
except:
    print("no internet... cannot download weights... ")
    

no internet... cannot download weights... 


In [10]:
# copy weights to torch cache

! cp ../input/vittiny/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz /root/.cache/torch/hub/checkpoints/

In [11]:

# predict
# create empty array
pred_all = np.zeros(len(test_df))

# loop over folds 
for i in range(5):
    print(f"Loading fold {i} weights and perform inference") # 
    model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"../input/vit-small/pawnet_lightning_vit_tiny_patch16_224/default/version_{i}/checkpoints/best_loss.ckpt",criterion=criterion)
    model = model.to("cuda")
    _,pred = inference_test(model,test_loader,criterion=None,device="cuda")
    pred_all += pred.numpy()

Loading fold 0 weights and perform inference
Loading fold 1 weights and perform inference
Loading fold 2 weights and perform inference
Loading fold 3 weights and perform inference
Loading fold 4 weights and perform inference


In [12]:
# create submission
sub = test_df[["Id"]]
sub["Pawpularity"] = pred_all / 5

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
sub.head()

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,43.270519
1,43a2262d7738e3d420d453815151079e,44.627859
2,4e429cead1848a298432a0acad014c9d,44.033482
3,80bc3ccafcc51b66303c2c263aa38486,46.359930
4,8f49844c382931444e68dffbe20228f4,47.919749


In [14]:
sub.to_csv("submission.csv", index=False)